# Interactive Mapping Lab - INSTRUCTOR SOLUTIONS
## CSC 2053 - Lab 13

**Note to Instructors:** This notebook contains complete solutions to all exercises in the interactive_mapping.ipynb lab.

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium import plugins

# Load MLB data for solutions (can change to any sport)
sport = "MLB"
url = f'https://raw.githubusercontent.com/CSC-2053-100-Fall25/python-datascience-template/main/{sport}.csv'
df = pd.read_csv(url)
df_clean = df.dropna(subset=['lat', 'lon'])

print(f"Loaded {len(df_clean)} stations")

---
# Exercise 1.1 Solution: Create Your First Map

**Task:** Create a map showing stations from your home state with markers showing callsign, frequency, and city.

In [ ]:
# SOLUTION

# 1. Filter for stations in Pennsylvania (or any state)
home_state = "PA"
home_stations = df_clean[df_clean['state'] == home_state]

print(f"Found {len(home_stations)} stations in {home_state}")

# 2. Create a map centered on those stations
m = folium.Map(
    location=[home_stations['lat'].mean(), home_stations['lon'].mean()],
    zoom_start=7  # Zoom in closer for a single state
)

# 3. Add markers for all stations in the state
for idx, row in home_stations.iterrows():
    # Popup shows: callsign, frequency, and city
    popup_text = f"{row['callsign']} - {row['frequency']} MHz<br>{row['city']}"
    
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=popup_text,
        tooltip=row['callsign']
    ).add_to(m)

# 4. Display the map
m

---
# Exercise 2.1 Solution: Custom Marker Styling

**Task:** Create a map with the top 100 most powerful stations featuring:
- Circle markers sized by power
- Colors based on format
- Rich HTML popups
- Tooltips with callsign and power

In [ ]:
# SOLUTION

# Filter for top 100 by power
top_power = df_clean.nlargest(100, 'erp')

print(f"Top 100 stations by power:")
print(f"  Power range: {top_power['erp'].min():.1f} - {top_power['erp'].max():.1f} kW")

# Define custom color scheme for formats
format_colors = {
    'Sports': 'red',
    'News/Talk': 'blue',
    'Country': 'green',
    'Classic Rock': 'purple',
    'Top 40': 'orange',
    'Classic Hits': 'darkblue',
    'Adult Contemporary': 'pink'
}

def get_format_color(format_name):
    """Return color for format, default to gray"""
    return format_colors.get(format_name, 'gray')

# Create the map
m = folium.Map(
    location=[top_power['lat'].mean(), top_power['lon'].mean()],
    zoom_start=4
)

# Normalize power for circle sizing
max_power = top_power['erp'].max()
min_power = top_power['erp'].min()

for idx, row in top_power.iterrows():
    # 1. Circle markers sized by power (5-20 range)
    power_ratio = (row['erp'] - min_power) / (max_power - min_power)
    radius = 5 + (power_ratio * 15)
    
    # 2. Colors based on format
    color = get_format_color(row['new_format'])
    
    # 3. Rich HTML popups showing all key information
    popup_html = f"""
    <div style="font-family: Arial; width: 220px;">
        <h3 style="margin-bottom: 5px; color: {color};">{row['callsign']}</h3>
        <p style="margin: 2px 0; font-size: 12px; color: #666;">High-Power Station</p>
        <hr style="margin: 5px 0;">
        <p style="margin: 3px 0;"><b>Frequency:</b> {row['frequency']} MHz</p>
        <p style="margin: 3px 0;"><b>Location:</b> {row['city']}, {row['state']}</p>
        <p style="margin: 3px 0;"><b>Format:</b> {row['new_format']}</p>
        <p style="margin: 3px 0;"><b>Power:</b> {row['erp']:.1f} kW</p>
        <p style="margin: 3px 0;"><b>Owner:</b> {row['owner']}</p>
        <p style="margin: 3px 0; font-size: 11px; color: #999;">
            Rank: #{idx+1} by power
        </p>
    </div>
    """
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        popup=folium.Popup(popup_html, max_width=300),
        # 4. Tooltips with callsign and power
        tooltip=f"{row['callsign']} ({row['erp']:.1f} kW)",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7,
        weight=2
    ).add_to(m)

# Add a legend/title
title_html = '''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 280px; height: 70px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; text-align: center; padding: 10px;">
        <h3 style="margin-bottom: 5px;">Top 100 Most Powerful Stations</h3>
        <p style="margin: 0; font-size: 12px;">Circle size = power level</p>
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

m

---
# Exercise 4.1 Solution: Advanced Visualization

**Task:** Combine multiple advanced features:
- Marker clusters for ALL stations
- Coverage circles for top 10 most powerful stations
- Different marker colors based on format
- Rich HTML popups

In [ ]:
# SOLUTION

# Create the base map
m = folium.Map(
    location=[df_clean['lat'].mean(), df_clean['lon'].mean()],
    zoom_start=4,
    tiles='CartoDB positron'
)

# 1. Marker clusters for ALL stations
marker_cluster = plugins.MarkerCluster(
    name='All Stations (Clustered)',
    show=True
).add_to(m)

# Add all stations to cluster with color-coded markers
for idx, row in df_clean.iterrows():
    color = get_format_color(row['new_format'])
    
    popup_html = f"""
    <div style="font-family: Arial; width: 200px;">
        <h4 style="color: {color}; margin-bottom: 5px;">{row['callsign']}</h4>
        <hr style="margin: 5px 0;">
        <p style="margin: 3px 0;"><b>Frequency:</b> {row['frequency']} MHz</p>
        <p style="margin: 3px 0;"><b>Location:</b> {row['city']}, {row['state']}</p>
        <p style="margin: 3px 0;"><b>Format:</b> {row['new_format']}</p>
        <p style="margin: 3px 0;"><b>Power:</b> {row['erp']:.1f} kW</p>
    </div>
    """
    
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=folium.Popup(popup_html, max_width=250),
        tooltip=row['callsign'],
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(marker_cluster)

# 2. Coverage circles for the top 10 most powerful stations
top_10_power = df_clean.nlargest(10, 'erp')

coverage_group = folium.FeatureGroup(name='Top 10 Coverage Areas', show=True)

for idx, row in top_10_power.iterrows():
    # Calculate approximate coverage radius (rough formula)
    # Higher power = larger coverage area
    radius_meters = np.sqrt(row['erp']) * 8000  # Approximate
    
    # Add coverage circle
    folium.Circle(
        location=[row['lat'], row['lon']],
        radius=radius_meters,
        popup=f"""<b>{row['callsign']}</b><br>
                  Power: {row['erp']:.1f} kW<br>
                  Coverage: ~{radius_meters/1000:.0f} km radius""",
        tooltip=f"{row['callsign']} coverage area",
        color='red',
        fill=True,
        fillOpacity=0.1,
        weight=2
    ).add_to(coverage_group)
    
    # Add prominent marker for the station
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"""<div style="width: 200px;">
                  <h3 style="color: red;">{row['callsign']}</h3>
                  <p><b>Power:</b> {row['erp']:.1f} kW</p>
                  <p><b>Location:</b> {row['city']}, {row['state']}</p>
                  <p><b>Format:</b> {row['new_format']}</p>
                  </div>""",
        tooltip=f"{row['callsign']} - {row['erp']:.0f} kW",
        icon=folium.Icon(color='red', icon='tower-broadcast', prefix='fa')
    ).add_to(coverage_group)

coverage_group.add_to(m)

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Add title
title_html = '''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 320px; height: 80px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; text-align: center; padding: 10px;">
        <h3 style="margin-bottom: 5px;">Advanced Station Visualization</h3>
        <p style="margin: 0; font-size: 12px;">Clusters + Coverage Areas + Custom Markers</p>
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

print("✓ Created advanced visualization with:")
print(f"  - {len(df_clean)} clustered stations")
print(f"  - {len(top_10_power)} coverage circles for high-power stations")
print(f"  - Color-coded by format")
print(f"  - Rich popups with station details")

m

---
# Exercise 5.1 Solution: Team Network Analysis

**Task:** Create a comprehensive map of a favorite team's affiliate network with:
- All affiliate stations
- Circle markers sized by power
- Coverage circles for the most powerful affiliates
- Rich popups with all station details
- A title banner identifying the team
- **Bonus:** Color-code by format

In [ ]:
# SOLUTION

# Pick a team (e.g., Yankees, Red Sox, Cardinals, etc.)
favorite_team = "New York Yankees"  # Change to any team in your dataset

# Get the team column name based on sport
team_column = sport.lower()  # 'mlb', 'nhl', 'nfl', or 'nba'

# Filter for this team's affiliates
team_affiliates = df_clean[df_clean[team_column] == favorite_team]

print(f"Analyzing {favorite_team} network:")
print(f"  Total affiliates: {len(team_affiliates)}")
print(f"  Power range: {team_affiliates['erp'].min():.1f} - {team_affiliates['erp'].max():.1f} kW")
print(f"  States covered: {team_affiliates['state'].nunique()}")

# Create the map centered on the network
m = folium.Map(
    location=[team_affiliates['lat'].mean(), team_affiliates['lon'].mean()],
    zoom_start=5,
    tiles='CartoDB positron'
)

# Get top 5 most powerful affiliates for coverage circles
top_affiliates = team_affiliates.nlargest(5, 'erp')

# Normalize power for circle sizing
max_power = team_affiliates['erp'].max()
min_power = team_affiliates['erp'].min()

# Add all affiliates with circle markers sized by power and color-coded by format
for idx, row in team_affiliates.iterrows():
    # Circle size based on power (5-15 range)
    if max_power > min_power:
        power_ratio = (row['erp'] - min_power) / (max_power - min_power)
    else:
        power_ratio = 0.5
    radius = 5 + (power_ratio * 10)
    
    # BONUS: Color-code by format
    color = get_format_color(row['new_format'])
    
    # Rich HTML popup
    popup_html = f"""
    <div style="width: 240px; font-family: Arial;">
        <h3 style="color: #003087; margin-bottom: 5px;">{row['callsign']}</h3>
        <p style="margin: 2px 0; font-size: 14px; font-weight: bold; color: #e74c3c;">
            {favorite_team} Affiliate
        </p>
        <hr style="margin: 5px 0;">
        <p style="margin: 3px 0;"><b>Frequency:</b> {row['frequency']} MHz</p>
        <p style="margin: 3px 0;"><b>Location:</b> {row['city']}, {row['state']}</p>
        <p style="margin: 3px 0;"><b>Format:</b> <span style="color: {color};">{row['new_format']}</span></p>
        <p style="margin: 3px 0;"><b>Power:</b> {row['erp']:.1f} kW</p>
        <p style="margin: 3px 0;"><b>Owner:</b> {row['owner']}</p>
    </div>
    """
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"{row['callsign']} - {row['city']} ({row['erp']:.1f} kW)",
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.7,
        weight=2
    ).add_to(m)

# Add coverage circles for top 5 most powerful affiliates
for idx, row in top_affiliates.iterrows():
    radius_meters = np.sqrt(row['erp']) * 8000
    
    folium.Circle(
        location=[row['lat'], row['lon']],
        radius=radius_meters,
        popup=f"{row['callsign']}<br>Coverage: ~{radius_meters/1000:.0f} km",
        tooltip=f"{row['callsign']} coverage",
        color='#003087',  # Team color
        fill=True,
        fillOpacity=0.05,
        weight=1,
        dashArray='5, 5'
    ).add_to(m)

# Title banner identifying the team
title_html = f'''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 340px; height: 90px; 
                background-color: white; border:2px solid #003087; z-index:9999; 
                font-size:14px; text-align: center; padding: 10px;">
        <h3 style="margin-bottom: 5px; color: #003087;">{favorite_team}</h3>
        <p style="margin: 0; font-size: 16px; font-weight: bold;">Radio Affiliate Network</p>
        <p style="margin: 5px 0; font-size: 12px; color: #666;">
            {len(team_affiliates)} stations • {team_affiliates['state'].nunique()} states
        </p>
        <p style="margin: 5px 0; font-size: 11px; color: #999;">
            Circle size = power | Color = format
        </p>
    </div>
'''
m.get_root().html.add_child(folium.Element(title_html))

print(f"\n✓ Created comprehensive {favorite_team} network map!")

m

---
## Additional Notes for Instructors

### Grading Rubric Suggestions:

**Exercise 1.1 (15 points):**
- Correct state filtering (5 pts)
- Map centered appropriately (3 pts)
- Markers with required info (callsign, frequency, city) (7 pts)

**Exercise 2.1 (25 points):**
- Top 100 power filtering (5 pts)
- Circle markers sized by power (7 pts)
- Color coding by format (5 pts)
- Rich HTML popups (5 pts)
- Tooltips with callsign/power (3 pts)

**Exercise 4.1 (30 points):**
- Marker clustering implemented (8 pts)
- Coverage circles for top stations (8 pts)
- Format-based colors (6 pts)
- Rich popups (5 pts)
- Overall map quality/aesthetics (3 pts)

**Exercise 5.1 (30 points):**
- Team network filtering (5 pts)
- Power-based circle sizing (7 pts)
- Coverage circles for top affiliates (7 pts)
- Rich popups with details (5 pts)
- Title banner (3 pts)
- Bonus: Format color-coding (+3 pts extra credit)

### Common Student Issues to Watch For:
1. Forgetting to use `.dropna()` for lat/lon
2. Not properly filtering data before creating maps
3. Hardcoding values instead of calculating dynamically
4. Missing closing tags in HTML popups
5. Not testing at different zoom levels

### Extension Ideas:
- Export maps to HTML files
- Add custom icons or logos
- Create animated time-series maps
- Integrate with GeoPandas for state boundaries
- Build a Flask web app to serve these maps